In [1]:
import pandas as pd

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df.head(10)

0         03/25/93 Total time of visit (in minutes):\n
1                       6/18/85 Primary Care Doctor:\n
2    sshe plans to move as of 7/8/71 In-Home Servic...
3                7 on 9/27/75 Audit C Score Current:\n
4    2/6/96 sleep studyPain Treatment Pain Level (N...
5                    .Per 7/06/79 Movement D/O note:\n
6    4, 5/18/78 Patient's thoughts about current su...
7    10/24/89 CPT Code: 90801 - Psychiatric Diagnos...
8                         3/7/86 SOS-10 Total Score:\n
9             (4/10/71)Score-1Audit C Score Current:\n
dtype: object

In [11]:
def date_sorter():
    import re
    import dateutil.parser
    from calendar import month_name
    from datetime import datetime
    
    # data cleaning
    df = pd.DataFrame(doc, columns=['text'])
    pattern = "[,.]? \d{4}|".join(month_name[1:]) + "[,.]? \d{4}";

    df['text'] = df['text'].apply(lambda x: x.strip('\n'))

    df['date'] = df['text'].apply(lambda x:re.findall(r'\d{1,2}\/\d{1,2}\/\d{2,4}|\d{1,2}\-\d{1,2}\-\d{2,4}|[A-Z][a-z]+\-\d{1,2}\-\d{4}|[A-Z][a-z]+[,.]? \d{2}[a-z]*,? \d{4}|\d{1,2} [A-Z][a-z,.]+ \d{4}|[A-Z][a-z]{2}[,.]? \d{4}|'+pattern+r'|\d{1,2}\/\d{4}|\d{4}', x))
    df['date'][271] = [df['date'][271][1]]
    df['date'] = df['date'].apply(lambda x: x[0])
    df['date'][461] = re.findall(r'\d{4}',df['date'][461])[0]
    df['date'][465] = re.findall(r'\d{4}',df['date'][465])[0]

    date_list = list(df['date'])

    i=0
    for year in date_list:
        if(re.match(r'\d{4}', year)) :
            date_list[i] = 'January 1, '+ date_list[i]
            year = date_list[i]
        elif (re.match(r'\d{1,2}\/\d{4}', year)) :
            date_split = year.split('/')
            date_list[i] = date_split[0] + '/1/'+ date_split[1]
            year = date_list[i]
        elif(re.match(r'[A-Z][a-z]+[,.]? \d{4}', year)) :
            date_split = year.split(' ')
            date_list[i] = date_split[0] + ' 1 '+ date_split[1]
            year = date_list[i]
        date_list[i] = dateutil.parser.parse(date_list[i]).strftime("%m/%d/%Y")
        i = i+1
    
    df['date'] = date_list
    fun = lambda date: datetime.strptime(date, "%m/%d/%Y")
    df['index'] = sorted(range(len(date_list)), key=lambda x : fun(date_list[x]))
    df.drop('text', axis=1,inplace=True)
    return pd.Series(list(df['index']))   

date_sorter()    

0        9
1       84
2        2
3       53
4       28
5      474
6      153
7       13
8      129
9       98
10     111
11     225
12      31
13     171
14     191
15     486
16     335
17     415
18      36
19     323
20     405
21     422
22     375
23     380
24     345
25      57
26     481
27     436
28     104
29     299
      ... 
470    220
471    208
472    243
473    139
474    320
475    383
476    244
477    286
478    480
479    431
480    279
481    198
482    381
483    463
484    366
485    255
486    439
487    401
488    475
489    257
490    152
491    235
492    464
493    253
494    231
495    427
496    141
497    186
498    161
499    413
Length: 500, dtype: int64